# Material Balances VIII

[What is Bond listening to right now?](https://youtu.be/0y8Q2PATVyI?si=0g54oLKbhWLnzuz3)

This lecture solves more Material Balance Example Problems. It shifts slightly from [Notebook 20](https://jqbond.github.io/CEN587/Notebooks/587-N20.html) in that it focuses on steady state operation of CSTRs, whereas Notebook 20 solves problems involving Batch and Plug Flow Reactors. Observe that the analysis of a CSTR at steady state involves solving algebraic equations rather than differential equations. In these examples, we will shift away from purely analytical solutions and begin applying numerical algorithms; these include the familar `scipy.optimize.newton()` for solving univariate problems and `scipy.optimize.root()` for solving systems of algebraic equations. If you need to review these methods, they are covered in [Notebook 03](https://jqbond.github.io/CEN587/Notebooks/587-N03.html) 

In [1]:
import numpy as np
import scipy.optimize as opt

(example-problem-01)=
## Example Problem 01 

```{note}
This is the CSTR analog of the PFR design problem that we explored already in [Example Problem 01 from Notebook 18](https://jqbond.github.io/CEN587/Notebooks/587-N18.html#example-problem-01). We are considering the same reaction in a CSTR so that we can begin to see both similarities and differences between the two types of flow reactors and their respective analyses.
```

Consider the following liquid-phase, irreversible, homogeneous reaction that is occurring in a perfectly-mixed CSTR. 

$$2A + 3B \rightarrow C$$

This reaction is first order in $A$ and first order in $B$. You may assume that the liquid phase has a constant density. Additional data are given below:

\begin{align*}
    k &= 37.2 \ \textrm{L} \ \textrm{mol}^{-1} \ \textrm{h}^{-1}\\
    Q_f &= 12 \ \textrm{L} \ \textrm{h}^{-1}\\
    C_{Af} &= 0.74 \ \textrm{mol} \ \textrm{L}^{-1}\\
    C_{Bf} &= 2.50 \ \textrm{mol} \ \textrm{L}^{-1}
\end{align*}

Find the CSTR volume required for a fractional conversion of 65% for species $A$.

**Answer**: $V_{CSTR} = 0.168 L$ 

```{important}
Recall the final answer from [Example Problem 01 from Notebook 18](https://jqbond.github.io/CEN587/Notebooks/587-N18.html#example-problem-01), where we concluded that a PFR volume of 0.0823L will give 65% conversion for this exact system. 	
```


### Solution to Example Problem 01

The problem asks us to solve for the CSTR volume required to achieve a certain conversion of the feed. This is a clear signal that we need to write material balances because they are the tools that describe the change in species quantities as a function of reactor size or reaction time. 

One important thing to recognize right away is that CSTRs at steady state will involve solving algebraic equations, so the strategies we use are different from Batch Reactors and Tubular Reactors, which both require solving differential equations. The general concepts and approaches are the same, but be mindful that there are different challenges associated with solving algebraic and differential equations. In general, with algebraic equations, it is usually advantageous to reduce the system to the smallest number of unknowns possible (instead of writing extra equations). Writing extra equations is very easy when we're solving ODEs numerically, but it can sometimes make things harder when we solve algebraic problems numerically. So, ***often*** (though unfortunately, I can't say always) we try to reduce the number of unknowns as much as possible by solving CSTR problems (at steady state) in terms of extents of reaction or fractional conversions.

Before we get to numerical methods, let's see if we can complete an analytical solution by hand using methods that are pretty familiar to everyone. We will begin by writing a balance on species $A$ in a perfectly-mixed CSTR operating at steady state:

$$0 = F_{A,f} - F_A + R_AV$$

Next, we expand the definitions of each term as needed to facilitate solution. First, we are given a feed volumetric flowrate, $Q_f$, and a feed concentration of $A$, $C_A$. We can use these to compute the feed molar flowrate of $A$ (which appears in our balance):

$$F_{A,f} = C_{A,f}Q_f$$

The balance equation also contains an intensive production rate, $R_A$, which we define as usual:

$$R_A = \sum_{i = 1}^{N_R} \nu_{i,A} \, r_i$$

Here, species A is participating in only one reaction, so:

$$R_A = -2r$$

We know the rate expression:

$$r = kC_AC_B$$

So this gives our production rate:

$$R_A = -2kC_AC_B$$

Which we can substitute into the material balance:

$$0 = F_{A,f} - F_A - 2kC_AC_BV$$

We have 4 unknowns in this equation: $F_A$, $C_A$, $C_B$, and $V$. However, we do know the exit conversion is 65%, so we can actually solve for $F_A$, $C_A$, $C_B$:

Specifically:

\begin{align*}
    F_A &= F_{A,f}(1 - X_A) \\
    F_B &= F_{B,f} - 3/2F_{A,f}X_A \\
    C_A &= F_A/Q \\
    C_B &= F_B/Q \\
\end{align*}

Since the density is constant in this flow reactor, we know that $Q = Q_f$, so everything on the right hand side here can be solved to give the values of $F_A$, $C_A$, and $C_B$ at 65% conversion. That leaves us one unknown in the material balance ($V$), so we can solve for it:

$$V = \frac{F_{A,f} - F_A}{2kC_AC_B}$$

Plugging in values, we get:

$$V = 0.168 \mathrm{L}$$

```{note}
Compare this result with the PFR Volume from [Example Problem 01 from Notebook 18](https://jqbond.github.io/CEN587/Lectures/587-L18.html#example-problem-01), where we found that a 0.0823L PFR is needed for a conversion of $X_A = 0.65$ for identical conditions. Why do we observe this difference between the two reactors?
```

In [2]:
############################################################################
# This cell uses Python like a calculator; we're just plugging in numbers  #
############################################################################

k   = 37.2  #L/mol/h
Qf  = 12.0 #L/h
Q   = Qf
CAf = 0.74 #mol/L
CBf = 2.50 #mol/L
FAf = CAf*Qf #mol/h
FBf = CBf*Qf #mol/h
XA  = 0.65
FA  = FAf*(1 - XA) #mol/h
FB  = FBf - 3/2*FAf*XA #mol/h
CA  = FA/Q
CB  = FB/Q
r   = k*CA*CB
RA  = -2*r
V   = -1*(FAf - FA)/RA
print(f'The CSTR volume required for XA = 0.65 is {V:5.3f}L')

The CSTR volume required for XA = 0.65 is 0.168L


(example-problem-02)=
## Example Problem 02 

```{note}
This is the CSTR analog of the PFR design problem that we explored already in [Example Problem 02 from Notebook 17](https://jqbond.github.io/CEN587/Notebooks/587-N17.html#example-problem-02). We are considering the same reaction in a CSTR so that we can begin to see both similarities and differences between the two types of flow reactors and their respective analyses.
```

Consider the following liquid-phase, irreversible, homogeneous reaction that is occurring in a perfectly-mixed CSTR. 

$$A + B \rightarrow C$$

This reaction is first order in $A$ and first order in $B$. You may assume that the liquid phase has a constant density. Additional data are given below:

\begin{align*}
    k &= 25.3 \ \textrm{L} \ \textrm{mol}^{-1} \ \textrm{h}^{-1}\\
    Q_f &= 10 \ \textrm{L} \ \textrm{h}^{-1}\\
    F_{Af} &= 10 \ \textrm{mol} \ \textrm{h}^{-1}\\
    F_{Bf} &= 10 \ \textrm{mol} \ \textrm{h}^{-1}
\end{align*}

What fractional conversion of species A is achieved in 1.24L CSTR? 

**Answer**: $X_A = 0.573$ 

```{important}
Recall the final answer from [Example Problem 02 from Notebook 17](https://jqbond.github.io/CEN587/Notebooks/587-N17.html#example-problem-02), where we concluded that a 1.24L PFR will achieve a conversion of $X_A = 0.758$. 	
```

### Solution to Example Problem 02

We are asked to relate the fractional conversion of $A$ to reactor size, which prompts us to write a balance on $A$:

$$0 = F_{A,f} - F_A + R_AV$$

The feed molar flowrate of $A$, $F_{A,f}$ is given in the problem statement $(F_{A,f} = 10 \ \mathrm{mol h^{-1}})$, as is the reactor volume $(V = 1.24\mathrm{L})$. We also see an intensive production rate, $R_A$, which we define as usual:

$$R_A = \sum_{i = 1}^{N_R} \nu_{i,A} \, r_i$$

For this single reaction:

$$R_A = -r$$

We know the rate expression:

$$r = kC_AC_B$$

So this gives

$$R_A = -kC_AC_B$$

Which we can substitute into the material balance:

$$0 = F_{A,f} - F_A - kC_AC_BV$$

At the moment, we have 3 unknowns in this equation: $F_A$, $C_A$, $C_B$. If we want to solve this by hand, we'll need to express $F_A$, $C_A$, $C_B$ in terms of a single unknown.  I'll choose to work with fractional conversion of $A$ here.  Completing a mole table, we find:

\begin{align}
    F_A &= F_{A,f}(1 - X_A) \\
    F_B &= F_{B,f} - F_{A,f}X_A \\
\end{align}

But: we see in the problem statement, that for this problem, $F_{A,f} = F_{B,F}$.  So, in this special case:

\begin{align}
    F_A &= F_{A,f}(1 - X_A) \\
    F_B &= F_{A,f}(1 - X_A) \\
\end{align}

Because this is a flow reactor operating at steady state, it is convenient to express concentrations in terms of flowrates. We further know that for a constant density process in a flow reactor, $Q = Q_f$:

\begin{align}
    C_A &= F_A/Q_f \\
    C_B &= F_A/Q_f \\
\end{align}

Substituting everything into the material balance, we get:

$$0 = F_{A,f} - F_{A,f}(1 - X_A) - kF_{A,f}^2(1-X_A)^2\frac{V}{Q_f^2}$$

We can solve this by simplifying, doing some FOIL magic, and using the quadratic formula. Ultimately, we get:

$$0 = X_A^2 + (\alpha - 2)X_A + 1$$

Where $\alpha = -Q_f^2/k/F_{A,f}/V$

If you solve that with the quadratic formula, you'll find two roots at:

$$X_A = 0.573$$

and at

$$X_A = 1.746$$

Mathematically, they are both fine, but physically, we can't have a conversion of more than 100% since it would mean we consume more reactant than we put into the reactor.  So we reject the upper root and conclude that our solution is:

$$X_A = 0.573$$

```{important}
Compare this to the final answer from [Example Problem 02 from Notebook 17](https://jqbond.github.io/CEN587/Notebooks/587-N17.html#example-problem-02), where we concluded that a 1.24L PFR will achieve a conversion of $X_A = 0.758$. Why do we observe this difference between the two reactors? 	
```

In [12]:
k   = 25.3 #L/mol/h
FAf = 10   #mol/h
Qf  = 10   #L/h
V   = 1.24 #L
α = -Qf**2/k/FAf/V
a = 1
b = α - 2
c = 1

Xupper = (-b + np.sqrt(b**2 - 4*a*c))/2
Xlower = (-b - np.sqrt(b**2 - 4*a*c))/2
print(Xlower, Xupper)

0.5727289521699684 1.7460266260526507


### Solving Example Problem 02 with Numerical Methods

Above, we found the closed-form, analytical solution to this problem using the quadratic formula. You can usually solve quadratics by hand, but for equations that are any more nonlinear than quadratic, you're probably going to use a numerical root finding algorithm. Further, in cases where you have more than one reaction, you'll will almost certaintly end up with more than one equation. If those are nonlinear, it is usually most efficient to solve them with a nonlinear algebraic equation solver for systems of equations, such as `scipy.optimize.root()`. Below, we'll walk through what I consider to be best practice when you are solving algebraic equations with a nonlinear equation solver. It is analogous to what I would consider best practice for solving ODEs, and the basic concept is that we are going to offload all of the substitutions and tedium to Python. As we will see below, this will generate a clean piece of code where each equation is physically significant and identifiable to us. This makes it much easier to find mistakes when they inevitably arise. As with all algebraic equation solvers, the basic approach is to write a relevant objective function, and then we'll have Python find the root of that function for us. Here, we only have a single uknown, $X_A$, so we'll set it up as a univariate function and use `opt.newton()` to solve it.

In [13]:
################################################################################
# Define a univariate objective function 0 = FAF - FA + RA*V                   #
################################################################################

def P02(XA):
    FAf = 10 #mol/h
    FBf = 10 #mol/h
    Qf  = 10 #L/h
    k   = 25.3 #L/mol/h
    V   = 1.24 #L
    
    FA  = FAf*(1-XA)
    FB  = FBf - FAf*XA
    
    Q   = Qf
    CA  = FA/Q
    CB  = FB/Q
    
    r   = k*CA*CB
    
    RA  = -r
    
    LHS = FAf - FA + RA*V
    return LHS

###################################################################
# Solve the Univariate Objective Function                         #
###################################################################

XAsol, info = opt.newton(P02, 0.5, full_output = True) 
print(f'A 1.24L CSTR will achieve a conversion of XA = {XAsol:5.3f}')

A 1.24L CSTR will achieve a conversion of XA = 0.573


#### Some observations

This is not something you need to remember *per se*, but it is worth being aware of, even if you don't quite get why yet. Hopefully, you'll notice this pattern as you solve more problems.

***For problems where we are solving ODEs (PFR and batch reactor problems)***, it is *slightly* easier to solve problems where the reactor volume or reaction time are given and you are solving for the corresponding fractional conversion. This is especially true when we solve them numerically using something like `solve_ivp()`. Basically, if we are given the exit volume (or exit time in a batch reactor), we won't need to interpolate to find the true solution since we know exactly the correct `tspan` or `vspan`.  

***When we are solving steady-state CSTR problems, which are algebraic equations*** it is usually quite a bit easier to solve problems where we are given the desired fractional conversion and asked to solve for volume.  It is less straightforward to solve problems where we are given volume and asked to solve for the corresponding fractional conversion.  This is because we often get algebraic equations that are nonlinear in fractional conversion or concentration in a CSTR.  If we are given the desired exit conversion, it is straightforward to plug into the nonlinear equation and solve for volume (that's what we did above in [Example Problem 01](#example-problem-01)). If we're given volume, we have to use a root finding algorithm to solve the nonlinear equation for the corresponding conversion (that's what we did in[ Example Problem 02](#example-problem-02)).

***In addition:***

For problems where we are solving ODEs (Batch Reactors, Plug Flow/Packed Bed Reactors, CSTRs not at steady state), it is generally easy (numerically) to just add more and more ODEs to our system and have the numerical algorithm solve the coupled system.  In other words, once we're using something like `solve_ivp()`, it is almost just as easy to solve 2 or 10 or 20 ODEs as it is to solve 1 ODE. While it is possible to express molar quantities and molar flowrates in terms of extents or conversions, it is not all important to do so unless we are trying to pursue a solution by hand.

For problems where we are solving algebraic equations (CSTRs at steady state), numerical solutions become increasingly difficult as we add more equations, for example, with `opt.root()`. There is no hard and fast rule on this, but for algebraic systems, it is generally helpful to reduce the number of equations by expressing mole quantities and molar flowrates in terms of extents of reaction or fractional conversion. This won't always be possible or necessary, but we should always keep in mind that reducing the number of unknowns in algebraic equations is generally useful, and we should try to do this if we can see straightforward ways to do so. 